In [8]:
import implicit
import pandas as pd
import numpy as np
from implicit import evaluation
from scipy import sparse

from src.conf import settings, COL_NAMES

In [ ]:
interactions = pd.read_parquet(settings.paths.interactions_preprocessed)

,item_id,favorite,score,status,progress,user_id
0,1,0,8,completed,26,1
1,1000,0,0,plan_to_watch,0,1
2,1002,0,0,plan_to_watch,0,1
3,1003,0,8,completed,3,1
4,1004,0,7,completed,1,1


In [9]:
interactions_completed = interactions[interactions[COL_NAMES.status] == "completed"]

In [12]:
interactions_completed[COL_NAMES.user_id] = interactions_completed[COL_NAMES.user_id].astype("category")
interactions_completed[COL_NAMES.item_id] = interactions_completed[COL_NAMES.item_id].astype("category")
user_id_cat = interactions_completed[COL_NAMES.user_id].cat.codes.values
item_id_cat = interactions_completed[COL_NAMES.item_id].cat.codes.values

C:\Users\epirt\AppData\Local\Temp\ipykernel_13468\3022476591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_completed[COL_NAMES.user_id] = interactions_completed[COL_NAMES.user_id].astype("category")
C:\Users\epirt\AppData\Local\Temp\ipykernel_13468\3022476591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_completed[COL_NAMES.item_id] = interactions_completed[COL_NAMES.item_id].astype("category")


In [ ]:
ratings = (
    (interactions_completed[COL_NAMES.score] +  interactions_completed[COL_NAMES.favorite]) * np.int16(1)
).values
ratings_csr = sparse.csr_matrix(
    (
        ratings,
        (
            user_id_cat,
            item_id_cat,
        ),
    )
)

In [ ]:
train_set, test_eval_set = evaluation.train_test_split(ratings_csr, random_state=42)

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=128, calculate_training_loss=True)
model.fit(train_set, show_progress=True)

In [ ]:
rating = evaluation.ranking_metrics_at_k(model, train_set, test_eval_set)

In [ ]:
item_factors = model.item_factors
embeddings_index = interactions_completed["track_id"].cat.categories
item_embeddings_df = pd.DataFrame(
    item_factors.to_numpy(),
    index=embeddings_index,
    columns=[f"als_{i}" for i in range(1, item_factors.shape[1] + 1)]
)
item_embeddings_df.head()